In [1]:
# ===== Reduced Order Modeling with NGSolve + pyMOR (POD-Galerkin) =====
from ngsolve import *
from ngsolve import Mesh
from ngsolve import x, y, z
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square
from netgen.occ import *

from pymor.algorithms.pod import pod
from pymor.vectorarrays.numpy import NumpyVectorSpace
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np

In [2]:
# ---------- Parameters ----------
L, W, H      = 3.0, 1.0, 1.0
baffle_thk   = 0.05
baffle_len   = 0.70*W
baffle_x     = [0.7, 1.5, 2.3]   # positions of the three baffles
half_model   = False
maxh         = 0.1

# stub parameters
Lin_ext  = 0.4   # inlet stub length
Lout_ext = 0.4   # outlet stub length
stub_frac = 0.5  # fraction of duct cross-section used for stub

# ---------- Main duct ----------
duct = Box(Pnt(0,0,0), Pnt(L, W, H))
duct.faces.name = "walls"

# ---------- Baffles ----------
fluid = duct
for i, x0 in enumerate(baffle_x, 1):
    x0min = x0 - 0.5*baffle_thk
    x0max = x0 + 0.5*baffle_thk
    zmin, zmax = 0, H

    if i == 2:  # middle baffle from TOP wall
        ymin, ymax = W - baffle_len, W
    else:       # side baffles from BOTTOM wall
        ymin, ymax = 0, baffle_len

    plate = Box(Pnt(x0min, ymin, zmin), Pnt(x0max, ymax, zmax))
    plate.faces.name = f"baffle{i}"
    fluid = fluid - plate

# ---------- Inlet stub ----------
stub_ymin = (1-stub_frac)/2 * W
stub_ymax = (1+stub_frac)/2 * W
stub_zmin = (1-stub_frac)/2 * H
stub_zmax = (1+stub_frac)/2 * H

stub_in = Box(Pnt(-Lin_ext, stub_ymin, stub_zmin),
              Pnt(0,        stub_ymax, stub_zmax))
stub_in.faces.name = "walls"           # stub sides are walls
stub_in.faces.Min(X).name = "inlet"    # only far end is inlet

# ---------- Outlet stub ----------
stub_out = Box(Pnt(L, stub_ymin, stub_zmin),
               Pnt(L+Lout_ext, stub_ymax, stub_zmax))
stub_out.faces.name = "walls"          # stub sides are walls
stub_out.faces.Max(X).name = "outlet"  # only far end is outlet

# ---------- Combine duct + stubs ----------
fluid = fluid + stub_in + stub_out

# ---------- Apply half-model cut if desired ----------
if half_model:
    sym_half = HalfSpace(Pnt(0, W/2, 0), Vec(0,1,0))
    fluid = fluid * sym_half

# ---------- Build OCC geometry and mesh ----------
geo = OCCGeometry(fluid)
m = geo.GenerateMesh(maxh=maxh)
mesh = Mesh(m)

print("Mesh elements:", mesh.ne)
print("Boundary names:", mesh.GetBoundaries())

# ---------- Count boundary elements ----------
counter = Counter()
for el in mesh.Elements(BND):
    counter[el.mat] += 1
print("Boundary facet counts:", counter)

# ---------- Visualize boundaries ----------
cf = mesh.BoundaryCF({
    "inlet":   10,
    "outlet":  20,
    "walls":   30,
    "baffle1": 40,
    "baffle2": 50,
    "baffle3": 60
}, default=0)

Draw(mesh)

Mesh elements: 16210
Boundary names: ('walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'baffle1', 'baffle2', 'baffle2', 'baffle2', 'walls', 'walls', 'walls', 'baffle3', 'baffle3', 'baffle3', 'walls', 'baffle1', 'baffle1', 'inlet', 'walls', 'walls', 'walls', 'walls', 'outlet')
Boundary facet counts: Counter({'walls': 3324, 'baffle2': 374, 'baffle1': 336, 'baffle3': 318, 'inlet': 56, 'outlet': 50})


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [3]:


SetNumThreads(1)

# ------------ parameters ------------
nu_f      = 0.01          # viscosity
U0        = 1.0           # target peak inlet speed
omega0    = 0.3           # under-relaxation
tol       = 1e-4          # Picard stopping criterion
maxit     = 50            # allow more iterations if needed
gamma_gd  = 0.7           # grad-div penalty
use_backflow = True

# ------------ FE spaces ------------
V = VectorH1(mesh, order=1, dirichlet="walls|baffle1|baffle2|baffle3|inlet")
Q = H1(mesh, order=1)
fes = V*Q
(u, p), (v, q) = fes.TnT()

# inlet profile helper
def inlet_cf(U0):
    prof = U0 * (1 - (2*y/W-1)**2) * (1 - (2*z/H-1)**2)
    return CoefficientFunction((prof, 0, 0))

# lifting (velocity only)
def lifting(U0):
    g = GridFunction(fes)
    g.components[0].Set(inlet_cf(U0), BND, definedon=mesh.Boundaries("inlet"))
    return g

# ------------ Stokes warm-start ------------
gS = lifting(U0)  # directly use full U0

aS = BilinearForm(fes, symmetric=True)
aS += nu_f * InnerProduct(Grad(u), Grad(v)) * dx
aS += (-div(v)*p - q*div(u)) * dx
aS += 1e-12*p*q*dx
aS += gamma_gd * div(u)*div(v) * dx

fS = LinearForm(fes)
aS.Assemble(); fS.Assemble()
rhs = fS.vec.CreateVector(); rhs.data = fS.vec - aS.mat*gS.vec
sol = GridFunction(fes)
inv = aS.mat.Inverse(fes.FreeDofs(), inverse="pardiso")
sol.vec.data = gS.vec + inv*rhs

u_vel = GridFunction(V); u_vel.vec.data = sol.components[0].vec
p_prs = GridFunction(Q); p_prs.vec.data = sol.components[1].vec
print("Stokes L2(u) =", float(Integrate(Norm(u_vel), mesh)))

# ------------ Oseen iteration (direct to U0) ------------
n = specialcf.normal(3)
g = lifting(U0)
uk = GridFunction(V); uk.vec.data = u_vel.vec
omega = omega0
last_rel = None

for it in range(1, maxit+1):
    aO = BilinearForm(fes, symmetric=False)
    aO += nu_f * InnerProduct(Grad(u), Grad(v)) * dx
    aO += InnerProduct(Grad(u)*uk, v) * dx
    aO += (-div(v)*p - q*div(u)) * dx
    aO += 1e-12*p*q*dx
    aO += gamma_gd * div(u)*div(v) * dx

    if use_backflow:
        un = InnerProduct(uk, n)
        neg_un = IfPos(-un, -un, 0.0)   # penalize only backflow
        aO += 0.5*neg_un*InnerProduct(u, v) * ds(definedon=mesh.Boundaries("outlet"))

    fO = LinearForm(fes)
    aO.Assemble(); fO.Assemble()
    rhs = fO.vec.CreateVector(); rhs.data = fO.vec - aO.mat*g.vec
    sol = GridFunction(fes)
    inv = aO.mat.Inverse(fes.FreeDofs(), inverse="pardiso")
    sol.vec.data = g.vec + inv*rhs

    du  = GridFunction(V); du.vec.data = sol.components[0].vec - u_vel.vec
    rel = Norm(du.vec) / max(Norm(sol.components[0].vec), 1e-30)
    print(f"  iter {it:2d}: rel = {rel:.3e}, omega={omega:.2f}")

    # under-relaxed update
    u_vel.vec.data = (1-omega)*u_vel.vec + omega*sol.components[0].vec
    p_prs.vec.data = (1-omega)*p_prs.vec + omega*sol.components[1].vec
    uk.vec.data    = u_vel.vec

    if rel < tol:
        print("  converged.")
        break

    if last_rel is not None and rel > 1.25*last_rel and omega > 0.18:
        omega *= 0.7
    last_rel = rel

# ------------ Rescale velocity to U0 ------------
inlet_area = Integrate(1, mesh, BND, definedon=mesh.Boundaries("inlet"))
avg_inlet  = Integrate(Norm(u_vel), mesh, BND, definedon=mesh.Boundaries("inlet")) / inlet_area
scale = U0 / avg_inlet
print(f"\nScaling velocity field by factor {scale}")
u_vel.vec.data *= scale

avg_inlet_new = Integrate(Norm(u_vel), mesh, BND, definedon=mesh.Boundaries("inlet")) / inlet_area
print("new avg |u| on inlet =", avg_inlet_new)

# convection field for scalar problem
b = CoefficientFunction((u_vel[0], u_vel[1], u_vel[2]))
VTKOutput(ma=mesh, coefs=[u_vel], names=["u"], filename="solution_navier", subdivision=2).Do()


Stokes L2(u) = 1.2110488780460822
  iter  1: rel = 2.730e-01, omega=0.30
  iter  2: rel = 2.068e-01, omega=0.30
  iter  3: rel = 1.566e-01, omega=0.30
  iter  4: rel = 1.188e-01, omega=0.30
  iter  5: rel = 9.061e-02, omega=0.30
  iter  6: rel = 6.961e-02, omega=0.30
  iter  7: rel = 5.383e-02, omega=0.30
  iter  8: rel = 4.183e-02, omega=0.30
  iter  9: rel = 3.257e-02, omega=0.30
  iter 10: rel = 2.536e-02, omega=0.30
  iter 11: rel = 1.973e-02, omega=0.30
  iter 12: rel = 1.535e-02, omega=0.30
  iter 13: rel = 1.198e-02, omega=0.30
  iter 14: rel = 9.397e-03, omega=0.30
  iter 15: rel = 7.441e-03, omega=0.30
  iter 16: rel = 5.956e-03, omega=0.30
  iter 17: rel = 4.819e-03, omega=0.30
  iter 18: rel = 3.930e-03, omega=0.30
  iter 19: rel = 3.219e-03, omega=0.30
  iter 20: rel = 2.636e-03, omega=0.30
  iter 21: rel = 2.150e-03, omega=0.30
  iter 22: rel = 1.743e-03, omega=0.30
  iter 23: rel = 1.401e-03, omega=0.30
  iter 24: rel = 1.116e-03, omega=0.30
  iter 25: rel = 8.805e-04, om

'solution_navier'

In [ ]:
# ------------------------------
# 1) Full-order model setup
# ------------------------------
def setup_full_model(mesh, u_vel, Pe: float,
                     dirichlet_names="walls|inlet|baffle1|baffle2|baffle3",
                     supg_C=1.0):
    """
    Build the stationary advection–diffusion operator with SUPG:
        -nu Δu + b·∇u = 0
    with Dirichlet on 'dirichlet_names' and natural outflow elsewhere.
    Returns a dict with V, A, f, inverse, lifting helper, etc.
    """
    nu = 1.0/Pe

    # FE space (scalar H1)
    V = H1(mesh, order=1, dirichlet=dirichlet_names)
    u, v = V.TnT()

    # convection field b from NS velocity
    b = CoefficientFunction((u_vel[0], u_vel[1], u_vel[2]))
    bnorm = Norm(b) + 1e-12

    # Bilinear form (diffusion + advection + SUPG)
    a = BilinearForm(V, symmetric=False)
    a += nu * InnerProduct(grad(u), grad(v)) * dx
    a += InnerProduct(b, grad(u)) * v * dx

    hK  = specialcf.mesh_size
    tau = supg_C * hK / bnorm                # simple, effective τ ~ h/|b|
    a += tau * (InnerProduct(b,grad(u))) * (InnerProduct(b,grad(v))) * dx

    f = LinearForm(V)                         # zero volume source by default

    # Assemble once
    a.Assemble()
    f.Assemble()

    A = a.mat
    invA = A.Inverse(V.FreeDofs(), inverse="pardiso")

    # helper: boundary-lifting GridFunction g from {name: value/None}
    def make_lifting(bc_vals: dict) -> GridFunction:
        """Create GridFunction g with boundary values; use None for Neumann parts."""
        g = GridFunction(V)
        names = list(mesh.GetBoundaries())
        vals  = [0.0 if bc_vals.get(nm, None) is None else float(bc_vals[nm]) for nm in names]
        g.Set(CoefficientFunction(vals), BND)
        return g

    # prototype vector for safe allocations in NumPy <-> NGSolve transfers
    proto = GridFunction(V).vec

    # A_times: apply A to a NumPy vector of size N
    def A_times(vec_np: np.ndarray) -> np.ndarray:
        x = proto.CreateVector(); y = proto.CreateVector()
        if len(vec_np) != len(x.FV().NumPy()):
            raise ValueError(f"Dimension mismatch: got {len(vec_np)} vs FE dim {len(x.FV().NumPy())}")
        x.FV().NumPy()[:] = vec_np
        y.data = A * x
        return y.FV().NumPy()

    # Full-order solve (given Dirichlet values)
    def solve_fom(g1, g2, g3) -> GridFunction:
        bc_vals = {"walls":0.0, "inlet":0.0, "baffle1":g1, "baffle2":g2, "baffle3":g3, "outlet":None}
        g = make_lifting(bc_vals)
        rhs = f.vec.CreateVector(); rhs.data = f.vec - A * g.vec
        uN  = GridFunction(V)
        uN.vec.data = g.vec + invA * rhs
        return uN

    return dict(V=V, A=A, f=f, invA=invA, make_lifting=make_lifting,
                A_times=A_times, solve_fom=solve_fom)


# --------- OFFLINE ---------
def build_rom(fom,                      # from setup_full_model(...)
                   g1_fixed, g2_train, g3_fixed,
                   pod_rtol=1e-6, pod_modes=None,
                   dirichlet_params=("baffle1","baffle2","baffle3")):
    """
    Build a ROM with *full* offline/online split:
      - POD basis from homogeneous snapshots w = u - g
      - A_rb = V^T A V (dense) and its inverse
      - Precompute reduced RHS blocks B_r[:,p] = V^T A psi_p for each Dirichlet parameter
      - Store psi_p (as NumPy arrays) for optional full reconstruction u = sum(mu_p psi_p) + V w_r
    Online cost: O(r^2 + r P). Full-field reconstruction (optional) is O(N r + N P).
    """

    V            = fom["V"]
    A            = fom["A"]
    A_times      = fom["A_times"]
    solve_fom    = fom["solve_fom"]
    make_lifting = fom["make_lifting"]

    # --- homogeneous snapshots w = u - g ---
    snaps = []
    for g2 in g2_train:
        g2v = float(g2)
        u_full = solve_fom(g1_fixed, g2v, g3_fixed)
        g      = make_lifting({"walls":0.0, "inlet":0.0,
                               "baffle1":g1_fixed, "baffle2":g2v, "baffle3":g3_fixed,
                               "outlet":None})
        w_np = u_full.vec.FV().NumPy() - g.vec.FV().NumPy()
        snaps.append(w_np.copy())

    N = len(snaps[0])
    space   = NumpyVectorSpace(N)
    snap_va = space.make_array(np.vstack(snaps))       # (nsnaps, N)

    RB, svals = pod(snap_va, rtol=pod_rtol, modes=pod_modes, l2_err=False)
    r = len(RB)
    Vr = np.column_stack([RB[i].to_numpy().ravel() for i in range(r)])   # N x r

    # Reduced operator A_rb and its inverse (dense, tiny)
    A_V  = np.column_stack([A_times(Vr[:, j]) for j in range(r)])        # N x r
    A_rb = Vr.T @ A_V                                                    # r x r
    A_rb_inv = np.linalg.inv(A_rb)                                       # r x r

    # --- build lifting basis psi_p for each Dirichlet parameter name ---
    # psi_p has boundary 1 on that baffle, 0 on other Dirichlet boundaries
    psi_np = []               # list of length P with N-vectors
    for name in dirichlet_params:
        g = make_lifting({"walls":0.0, "inlet":0.0,
                          "baffle1": 1.0 if name=="baffle1" else 0.0,
                          "baffle2": 1.0 if name=="baffle2" else 0.0,
                          "baffle3": 1.0 if name=="baffle3" else 0.0,
                          "outlet":None})
        psi_np.append(g.vec.FV().NumPy().copy())

    # Precompute reduced RHS blocks: B_r[:,p] = V^T A psi_p
    P = len(dirichlet_params)
    Br = np.zeros((r, P))
    for p in range(P):
        Apsi = A_times(psi_np[p])       # N
        Br[:, p] = Vr.T @ Apsi          # r

    energy = float(np.sum(svals**2))
    def energy_captured(k): return float(np.sum(svals[:k]**2) / (energy+1e-30))

    # Package everything for the online step
    return dict(V=fom["V"], Vr=Vr, r=r,
                A_rb=A_rb, A_rb_inv=A_rb_inv, Br=Br,
                psi_np=np.column_stack(psi_np),            # N x P
                param_names=list(dirichlet_params),
                energy_total=energy, svals=svals, energy_captured=energy_captured)


# --------- ONLINE ---------
def solve_rom(rom, mu_dict, return_full=True):
    """
    Solve reduced system given parameter dict mu_dict for the baffles, e.g.:
      mu_dict = {"baffle1": g1, "baffle2": g2, "baffle3": g3}
    If return_full=False, returns only (w_r, uhat_r) where uhat_r are the reduced coefficients of u.
    """
    Vr       = rom["Vr"]
    r        = rom["r"]
    A_rb_inv = rom["A_rb_inv"]
    Br       = rom["Br"]          # r x P
    names    = rom["param_names"]

    # build parameter vector μ = [g_{baffle1}, g_{baffle2}, g_{baffle3}]^T
    mu = np.array([float(mu_dict.get(nm, 0.0)) for nm in names])    # (P,)

    # rhs_rb = - sum_p mu_p * Br[:,p]
    rhs_rb = - Br @ mu                                             # (r,)

    # solve reduced system
    w_r = A_rb_inv @ rhs_rb                                        # (r,)

    if not return_full:
        return w_r
    

    # (optional) full reconstruction:
    # g(μ) = sum_p mu_p * psi_p  (N vector)  and  u = g + V * w_r
    psi_np = rom["psi_np"]                                         # N x P
    g_np   = psi_np @ mu                                           # N
    u_np   = g_np + Vr @ w_r                                       # N

    u = GridFunction(rom["V"])
    u.vec.FV().NumPy()[:] = u_np
    return u

# ------------------------------
# Example usage with fast ROM
# ------------------------------

# 1) Full-order operator setup
Pe = 400.0
fom = setup_full_model(mesh, u_vel, Pe=Pe, supg_C=1.0)

# 2) Offline ROM construction
g1_fix, g3_fix = 6.0, 12.0
g2_train = np.linspace(1, 12, 14)    # 14 snapshots
rom = build_rom(fom, g1_fixed=g1_fix, g2_train=g2_train, g3_fixed=g3_fix,
                     pod_rtol=1e-3, pod_modes=None)

print(f"POD kept r={rom['r']} modes; "
      f"Energy captured={rom['energy_captured'](rom['r']):.5f}")

# 3) Compare FOM vs ROM for a test parameter
g1t, g2t, g3t = 6.0, 9.0, 12.0

import time
start = time.time()
for i in range(100):
    u_fom = fom["solve_fom"](g1t, g2t, g3t)
end = time.time()
print("FOM time (100 solves) =", end-start)

start = time.time()
for i in range(100):
    u_rom = solve_rom(rom, {"baffle1": g1t, "baffle2": g2t, "baffle3": g3t})
end = time.time()
print("ROM time (100 solves) =", end-start)

# 4) Errors
A = fom["A"]
diff = GridFunction(fom["V"]); diff.vec.data = u_fom.vec - u_rom.vec
num = InnerProduct(A * diff.vec, diff.vec)
den = max(InnerProduct(A * u_fom.vec, u_fom.vec), 1e-30)
print(f"Relative A-energy error ≈ {float(np.sqrt(num/den)):.3e}")

errL2  = sqrt( Integrate((u_fom - u_rom)**2, mesh) )
normL2 = sqrt( Integrate(u_fom**2,              mesh) )
print("rel L2 =", float(errL2/normL2))

# 5) Export both to ParaView
VTKOutput(ma=mesh, coefs=[u_fom, u_rom], names=["u_fom", "u_rom"],
          filename="rom_compare", subdivision=2).Do()

# -- Sanity check: homogeneous snapshot check --
g2_chk = float(g2_train[len(g2_train)//2])
u_fom_chk = fom["solve_fom"](g1_fix, g2_chk, g3_fix)
g_chk = fom["make_lifting"]({"walls":0.0, "inlet":0.0,
                             "baffle1":g1_fix, "baffle2":g2_chk, "baffle3":g3_fix,
                             "outlet":None})
w_true_np = u_fom_chk.vec.FV().NumPy() - g_chk.vec.FV().NumPy()


Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

POD kept r=2 modes; Energy captured=1.00000
FOM time (100 solves) = 0.6381721496582031
ROM time (100 solves) = 0.004998445510864258
Relative A-energy error ≈ 5.808e-16
rel L2 = 1.9504181870964284e-16
